In [26]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

from torchvision import datasets

from tqdm import tqdm
import pickle

import pandas as pd

In [2]:

url = 'https://www.cs.toronto.edu/~kriz/cifar-10-sample/dog10.png'
image = Image.open(requests.get(url, stream=True).raw)
feature_extractor = ViTFeatureExtractor.from_pretrained('nateraw/vit-base-patch16-224-cifar10')
model = ViTForImageClassification.from_pretrained('nateraw/vit-base-patch16-224-cifar10')
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
preds = outputs.logits.argmax(dim=1)

classes = [
    'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
]
classes[preds[0]]

/Users/polaris000/Library/Python/3.9/lib/python/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


'dog'

In [3]:
import sys
sys.path.append("..")
sys.path.append("../miniresnet")

In [4]:
from miniresnet.process import load_data

In [5]:
train_loader, val_loader, test_loader, _ = load_data((3, 32, 32))

Files already downloaded and verified
Files already downloaded and verified


In [5]:
val_set = datasets.CIFAR10(
        root="../data", train=False, download=True
    )

Files already downloaded and verified


In [9]:
correct = 0
total = 0

for images, labels in tqdm(val_set):
    images = feature_extractor(images=images, return_tensors="pt")
    outputs = model(**images)
    preds = outputs.logits.argmax(dim=1)
    total += 1
    if preds[0] == labels:
        correct += 1

print(f"Accuracy: {correct/total}")

100%|██████████| 10000/10000 [15:25<00:00, 10.81it/s]

Accuracy: 0.9852


In [11]:
with open("../data/testdata/cifar_test_nolabels.pkl", "rb") as f:
    data = pickle.load(f)

In [13]:
data.keys()

dict_keys([b'data', b'ids'])

In [17]:
# Predict labels and add as a third field
data[b"label"] = []
for img in tqdm(data[b"data"]):
    image = img.reshape(3, 32, 32).transpose(1, 2, 0)
    inputs = feature_extractor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    preds = outputs.logits.argmax(dim=1)
    data[b'label'].append(preds[0].item())

# Save the pickle file
with open("../data/testdata/cifar_test_labelled.pkl", "wb") as f:
    pickle.dump(data, f)


100%|██████████| 10000/10000 [19:45<00:00,  8.44it/s]


In [24]:
data[b"label"] = [datapoint.item() for datapoint in data[b"label"]]

In [30]:
res2 = pd.read_csv("../data/results/results_v2.csv")

In [32]:
res2["ground_truth"] = data[b"label"]

In [33]:
res2

,ID,Labels,ground_truth
0,0,8,8
1,1,8,8
2,2,8,8
3,3,8,8
4,4,8,8
...,...,...,...
9995,9995,5,5
9996,9996,5,5
9997,9997,3,3
9998,9998,5,5


In [38]:
print("Approx acc: ", (res2["ground_truth"] == res2["Labels"]).sum() / len(res2))

Approx acc:  0.7692
